# Notebook de Carga de Datos iniciales

Este notebook esta destinado a la carga inicial de los datos en los distintas base de datos

## Importación de librerias

In [1]:
from pathlib import Path
from db_connections import client,db_neo4j,db_redis
from src import mongo,neo4j,utils,redis
from pymongo.errors import ConnectionFailure
import os

## Conexión con bases de datos

In [2]:
# =====================
# TEST DE CONEXIONES
# =====================

#MongoDB
try:
    client.admin.command("ping")  # fuerza conexión al servidor
    print("✅ Conexión a MongoDB verificada.")
except ConnectionFailure as e:
    print(f"❌ Falló la conexión: {type(e).__name__} - {e}")

#Neo4j
try:
    db_neo4j.verify_connectivity()
    print("✅ Conexión a Neo4j verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

#Redis
try:
    db_redis.ping()
    print("✅ Conexión a Redis verificada.")
except Exception as e:
    print(f"❌ Error de conexión: {type(e).__name__} - {e}")

✅ Conexión a MongoDB verificada.
✅ Conexión a Neo4j verificada.
✅ Conexión a Redis verificada.


## Carga de Datos

In [10]:
# --- MONGO ---
nombre_base="clase"
nombre_colecciones = ["usuarios","destinos","hoteles","reservas","actividades"]
nombre_archivos= ["usuarios.csv","destinos.csv","hoteles.csv","reservas.csv","actividades.csv"]

for i in range (5):
    ruta=Path("fuentes") / nombre_archivos[i]
    df=utils.lectura_csv(ruta)
    
    
    if df is not None:
        filas = df.to_dict(orient="records")
        
        # Inserta datos en colecciones
        mongo.crear_coleccion(nombre_base,nombre_colecciones[i],recrear=True)
        mongo.insertar_muchos_coleccion(nombre_base,nombre_colecciones[i],filas)

        if nombre_colecciones[i]=="usuarios":
            conectados=redis.guardar_usuarios_conectados(df,15)
            print(f"✅ Se registran {conectados} usuarios conectados")
        
        # Crea nodos en Neo4j solo para usuarios y destinos
        if nombre_colecciones[i] in ["usuarios", "destinos"]:
            nombre_nodo = "Usuario" if nombre_colecciones[i] == "usuarios" else "Destino"
            campo_clave = "usuario_id" if nombre_nodo == "Usuario" else "destino_id"

            with db_neo4j.session() as session:
                filas = df.to_dict(orient="records")
                for fila in filas:
                    nodo = session.execute_write(neo4j.crear_nodo, nombre_nodo, campo_clave, fila)
                print(f"✅ Nodos de {nombre_nodo} creados exitosamente")
        
        if nombre_colecciones[i] == "reservas":
            
            # Crear relaciones VISITO       
            with db_neo4j.session() as session: 
                filas = df.to_dict(orient="records")
                for fila in filas:
                    session.execute_write(
                        neo4j.crear_relacion,
                            "Usuario", "usuario_id",fila["usuario_id"],
                            "Destino", "destino_id",fila["destino_id"],"VISITO")
                print("✅ Relaciones VISITO creadas exitosamente")

            # Carga en Redis de reservas temporales
            df_reservas_temporales=df[df["estado"] == "Pendiente"]
            resultado=redis.carga_masiva_reservas_temporales(df_reservas_temporales)
            print(f"✅ Se insertaron {resultado} reservas temporales")
                 

✅ Se insertaron 1000 documentos en 'usuarios'.
✅ Se registran 15 usuarios conectados
✅ Nodos de Usuario creados exitosamente
✅ Se insertaron 50 documentos en 'destinos'.
✅ Nodos de Destino creados exitosamente
✅ Se insertaron 100 documentos en 'hoteles'.
✅ Se insertaron 200 documentos en 'reservas'.
✅ Relaciones VISITO creadas exitosamente
✅ Se insertaron 52 reservas temporales
✅ Se insertaron 120 documentos en 'actividades'.


##### Contar la cantidad de documentos existentes en cada colección

In [4]:
for i in range (5):
    cantidad=mongo.contar_documentos(nombre_base,nombre_colecciones[i])
    print(f"La cantidad de documentos de la coleccion {nombre_colecciones[i]} es {cantidad}")

La cantidad de documentos de la coleccion usuarios es 1000
La cantidad de documentos de la coleccion destinos es 50
La cantidad de documentos de la coleccion hoteles es 100
La cantidad de documentos de la coleccion reservas es 200
La cantidad de documentos de la coleccion actividades es 120


##### Listar datos de las colecciones

In [5]:
# Lista el primer dato de cada colección como prueba de la carga

for i in range (5):
    cursor=mongo.obtener_cursor(nombre_base,nombre_colecciones[i], 1,proyeccion={"_id":0})
    print(f"Colección {nombre_colecciones[i]}:")
    mongo.imprimir_cursor(cursor)

Colección usuarios:
{'apellido': 'Aguiló',
 'email': 'jose-ignaciocalatayud@gmail.com',
 'nombre': 'Brígida',
 'telefono': '+34 701 95 78 02',
 'usuario_id': 1}
Colección destinos:
{'ciudad': 'Valencia',
 'destino_id': 1,
 'pais': 'Brasil',
 'precio_promedio': 175291,
 'tipo': 'Playa'}
Colección hoteles:
{'calificacion': 4,
 'ciudad': 'Guadalajara',
 'hotel_id': 1,
 'nombre': 'Fábrica JJH S.Coop.',
 'precio': 218005,
 'servicios': 'pileta;wifi;spa'}
Colección reservas:
{'destino_id': 15,
 'estado': 'Cancelada',
 'fecha_reserva': '2025-08-07',
 'precio_total': 59359,
 'reserva_id': 1,
 'usuario_id': 319}
Colección actividades:
{'actividad_id': 1,
 'ciudad': 'Valladolid',
 'nombre': 'Organic object-oriented Internet solution',
 'precio': 78337,
 'tipo': 'aventura'}
